<a href="https://colab.research.google.com/github/trefftzc/partition_COLAB_notebooks/blob/main/partition_omp_for_gpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OpenMP for GPUs

OpenMP can be used to program GPUs.

The key pragmas that can be used to parallelize code so that it uses a GPU are:

#pragma omp target

which indicates to the compiler that the developer wants to
use the GPU for the code that follows and

#pragma omp loop

loop tells the compiler that the for statement that follows
should be executed on the GPU.

In [7]:
%%writefile partition_omp_gpu.c
/*
 *
 * Solve the Partition problem
 * https://en.wikipedia.org/wiki/Partition_problem
 * This code works for multisets of up to 32 elements
 * The input is expected to be as follows:
 * The first line will contain n, the number of elements in the multiset
 * The remaining n lines will contain the n values, one per line
 */
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

// This functions evaluate a partition of the multiset.
// The values in the multiset are passed as a parameter in array
// n is the size of the multiset
// value is an integer value. The binary code of value
// encodes a partition:
// One partition corresponds to the bits that are 0 in the binary encoding of value
// The other partition corresponds to the bits that are 1 in the binary encoding of value
int evaluatePartition(  int value, int n, int *array) {
  int sum0s = 0;
  int sum1s = 0;
  unsigned int mask = 1;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      sum1s = sum1s + array[i];
    }
    else {
      sum0s = sum0s + array[i];
    }
    mask = mask * 2;
  }
  if (sum0s == sum1s)
     return value;
  else
     return 0;
}

void printResults(unsigned int value,int n,int *array)
{
  printf("Solution:\n");
  printf("First partition: ") ;
  unsigned int mask = 1;
  int sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) != 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
  printf("Second partition: ") ;
  mask = 1;
  sum = 0;
  for(int i = 0;i < n;i++) {
    if ((mask & value) == 0) {
      printf("%d ",array[i]);
      sum = sum + array[i];
    }
    mask = mask * 2;
  }
  printf(" sum: %d \n",sum);
}

// The main function
int main() {

  int n;
  int *array;
// The format of the input is
// an integer value n with the size of the multiset
// n integer values with the multiset
  scanf("%d",&n);

  printf("The value of n is %d\n",n);
  array = (int *) malloc (n * sizeof(int));
  for(int i = 0;i < n;i++) {
    scanf("%d",&array[i]);
  }
  printf("The read values are: \n");
  for(int i = 0;i < n;i++) {
    printf("%d ",array[i]);
  }
  printf("\n");
// Calculate the set of the power set
  unsigned int nPartitions = 1;
  for(int i = 0;i < n;i++) {
    nPartitions = nPartitions * 2;
  }
  // printf("The number of possible partitions is: %d\n",nPartitions);
  // Only half of all possible partitions need be examined
  // The second half is symmetrical to the first half
  nPartitions = nPartitions / 2;
  int solutionFound = 0;
  int solution = -1;
  int *results;
  results = (int *) malloc (nPartitions*sizeof(int));
  #pragma omp target
  #pragma omp loop
  for(int i = 1;i < (nPartitions);i++) {
    results[i] = evaluatePartition(i,n,array);
  }

  for(int i = 0;i < nPartitions;i++) {
    if (results[i] > 0) {
      solutionFound = 1;
      solution = i;
    }
  }
  if (solutionFound) {
    printResults(solution, n, array);
  }
  else {
    printf("No solution was found.");
  }
  return 0;
}


Overwriting partition_omp_gpu.c


In [8]:
!cc partition_omp_gpu.c -o partition_omp_gpu -fopenmp -O3

partition_omp_gpu.c: In function ‘main’:
partition_omp_gpu.c:75:3: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   75 |   scanf("%d",&n);
      |   ^~~~~~~~~~~~~~
partition_omp_gpu.c:80:5: warning: ignoring return value of ‘scanf’ declared with attribute ‘warn_unused_result’ []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-result-Wunused-result]8;;]
   80 |     scanf("%d",&array[i]);
      |     ^~~~~~~~~~~~~~~~~~~~~


In [9]:
%%writefile test_with_solution_24.txt
24
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 23

Overwriting test_with_solution_24.txt


In [10]:
!time ./partition_omp_gpu < test_with_solution_24.txt

The value of n is 24
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 23 
Solution:
First partition: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1  sum: 23 
Second partition: 23  sum: 23 

real	0m0.557s
user	0m0.533s
sys	0m0.021s


In [11]:
%%writefile test_with_no_solution_24.txt
24
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 32

Overwriting test_with_no_solution_24.txt


In [12]:
!time ./partition_omp_gpu < test_with_no_solution_24.txt

The value of n is 24
The read values are: 
1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 32 
No solution was found.
real	0m0.556s
user	0m0.532s
sys	0m0.022s
